In [ ]:
import math
import random
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [ ]:
class MLP:
    def __init__(self, input_neurons, hidden_neurons, output_neurons):
        self.input_neurons = input_neurons
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons

        self.hidden_bias = []
        self.output_bias = []

        self.input_values = []
        self.hidden_values = []
        self.target_values = []
        self.output_values = []

        for _ in range(self.input_neurons):
            self.input_values.append(random.randint(1, 5))

        for _ in range(self.hidden_neurons):
            self.hidden_bias.append(round(random.random(), 2))

        for _ in range(self.output_neurons):
            self.output_bias.append(round(random.random(), 2))

        self.input_hidden_weights = []
        self.hidden_output_weights = []

        for e in range(self.input_neurons):
            self.input_hidden_weights.append([])
            for _ in range(self.hidden_neurons):
                self.input_hidden_weights[e].append(round(random.random(), 2))

        for e in range(self.hidden_neurons):
            self.hidden_output_weights.append([])
            for _ in range(self.output_neurons):
                self.hidden_output_weights[e].append(round(random.random(), 2))

    def forward_prop(self):
        self.hidden_values = []  # Clear hidden values
        self.output_values = []  # Clear output values

        for h in range(self.hidden_neurons):
            result = self.hidden_bias[h]
            for i in range(self.input_neurons):
                result += (self.input_values[i] * self.input_hidden_weights[i][h])
            self.hidden_values.append(result)

        for o in range(self.output_neurons):
            result = self.output_bias[o]
            for h in range(self.hidden_neurons):
                result += self.sigmoid_func(self.hidden_values[h]) * self.hidden_output_weights[h][o]
            self.output_values.append(result)

    def backward_prop(self, learning_rate):
        error_values = self.error_calc()

        self.output_new = []
        self.hidden_new = []

        self.hidden_output_new = []
        self.input_hidden_new = []

        for o in range(self.output_neurons):
            result = self.output_delta(error_values[o], self.output_values[o])
            self.output_new.append(result)

        for h in range(self.hidden_neurons):
            self.hidden_output_new.append([])
            for o in range(self.output_neurons):
                result = self.hidden_output_weights[h][o] + (learning_rate * self.hidden_values[h] * self.output_new[o])
                self.hidden_output_new.append(result)

        for h in range(self.hidden_neurons):
            result = 0
            for o in range(self.output_neurons):
                result += self.hidden_delta(self.output_new[o], self.hidden_output_weights[h][o], self.hidden_values[h])
            self.hidden_new.append(result)

        for i in range(self.input_neurons):
            self.input_hidden_new.append([])
            for h in range(self.hidden_neurons):
                result = self.input_hidden_weights[i][h] + (learning_rate * self.input_values[i] * self.hidden_new[h])
                self.input_hidden_new.append(result)

    def sigmoid_func(self, x):
        return 1 / (1 + pow(math.e, -x))

    def prime_func(self, x):
        return x * (1 - x)

    def error_calc(self):
        error_values = []
        for e in range(len(self.output_values)):
            error_values.append(abs(self.output_values[e] - self.target_values[e]))
        return error_values

    def output_delta(self, error, x):
        return error * self.prime_func(x)

    def hidden_delta(self, delta, weight, x):
        return delta * weight * x

    def calculate_loss(self):
        loss = 0
        for i in range(len(self.output_values)):
            loss += -self.target_values[i] * math.log(self.output_values[i] + 1e-15)  # Adding a small value to prevent log(0)
        return loss

In [ ]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Normalize the input data
X = (X - X.mean()) / X.std()

# Convert target labels to one-hot encoding
num_classes = len(np.unique(y))
y_onehot = np.eye(num_classes)[y]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Initialize the MLP with the appropriate number of input, hidden, and output neurons
mlp = MLP(input_neurons=X.shape[1], hidden_neurons=5, output_neurons=num_classes)

# Train the MLP using the training data
epochs = 1000
learning_rate = 0.1
for epoch in range(epochs):
    for i in range(len(X_train)):
        mlp.input_values = X_train[i]
        mlp.target_values = y_train[i]  # Set target values for each training example
        mlp.forward_prop()
        mlp.backward_prop(learning_rate)

# Test the MLP using the testing data
correct = 0
total = len(X_test)
for i in range(len(X_test)):
    mlp.input_values = X_test[i]
    mlp.forward_prop()
    predicted = np.argmax(mlp.output_values)
    actual = np.argmax(y_test[i])
    if predicted == actual:
        correct += 1

# Calculate accuracy
accuracy = correct / total
print(f"Accuracy: {accuracy}")

# Calculate loss
total_loss = 0
for i in range(len(X_test)):
    mlp.input_values = X_test[i]
    mlp.target_values = y_test[i]
    mlp.forward_prop()
    total_loss += mlp.calculate_loss()

average_loss = total_loss / len(X_test)
print(f"Average Loss: {average_loss}")


Accuracy: 0.36666666666666664
Average Loss: -0.6532694417695245
